In [8]:
# -------------------------------
# 1. Import Necessary Libraries
# -------------------------------
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import os
import tensorflow as tf

# Suppress TensorFlow and Keras warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel('ERROR')

# -------------------------------
# 2. Data Preparation
# -------------------------------

# Load data
X = np.load('Datasets/kryptonite-24-X.npy')
y = np.load('Datasets/kryptonite-24-y.npy')
X = np.round(X)

# Split into training (70%), validation (15%), and test sets (15%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print("Training set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)
print("Test set:", X_test.shape, y_test.shape)

# -------------------------------
# 3. Feature Scaling
# -------------------------------

# Standardize features to have zero mean and unit variance
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("Scaled features using StandardScaler.")

# -------------------------------
# 4. Conditional Autoencoder Setup
# -------------------------------

# One-hot encode labels
y_train_onehot = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_val_onehot = tf.keras.utils.to_categorical(y_val, num_classes=2)
y_test_onehot = tf.keras.utils.to_categorical(y_test, num_classes=2)

# Define input dimensions
input_dim = X_train_scaled.shape[1]  # Should be 24
encoding_dim = 2  # For 2D visualization

# Inputs for features and labels
input_features = Input(shape=(input_dim,), name='features_input')
input_labels = Input(shape=(2,), name='labels_input')  # Assuming binary classification

# Concatenate features and labels for the encoder
merged_input = concatenate([input_features, input_labels])

# Encoder network
encoded = Dense(64, activation='relu')(merged_input)
encoded = BatchNormalization()(encoded)
encoded = Dropout(0.2)(encoded)

encoded = Dense(32, activation='relu')(encoded)
encoded = BatchNormalization()(encoded)
encoded = Dropout(0.2)(encoded)

encoded_output = Dense(encoding_dim, activation='relu', name='encoded_output')(encoded)

# Concatenate encoded representation with labels for the decoder
decoder_input = concatenate([encoded_output, input_labels])

# Decoder network
decoded = Dense(32, activation='relu')(decoder_input)
decoded = BatchNormalization()(decoded)
decoded = Dropout(0.2)(decoded)

decoded = Dense(64, activation='relu')(decoded)
decoded = BatchNormalization()(decoded)

# Define the final decoded output with 24 units to match the shape of X
decoded_output = Dense(input_dim, activation='sigmoid', name='decoded_output')(decoded)

# Classification Head
classifier = Dense(16, activation='relu')(encoded_output)
classifier = BatchNormalization()(classifier)
classifier = Dropout(0.2)(classifier)
classifier_output = Dense(2, activation='softmax', name='classifier_output')(classifier)

# Define the combined model
model = Model(inputs=[input_features, input_labels], outputs=[decoded_output, classifier_output])

# Compile the model with correct loss functions and target-output matching
model.compile(optimizer='adam',
              loss={'decoded_output': 'mean_squared_error',  # Reconstruction loss for continuous data
                    'classifier_output': 'categorical_crossentropy'},  # Classification loss
              loss_weights={'decoded_output': 1.0, 'classifier_output': 1.0},
              metrics={'classifier_output': 'accuracy'})

# Define Early Stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# -------------------------------
# 5. Train the Conditional Autoencoder with explicit output matching
# -------------------------------

history = model.fit(
    [X_train_scaled, y_train_onehot],  # Input: Features and labels for concatenation
    {'decoded_output': X_train_scaled, 'classifier_output': y_train_onehot},  # Targets for each output
    epochs=100,
    batch_size=256,
    shuffle=True,
    validation_data=([X_val_scaled, y_val_onehot], {'decoded_output': X_val_scaled, 'classifier_output': y_val_onehot}),
    callbacks=[early_stop],
    verbose=1
)

# -------------------------------
# 6. Extract 2D Embeddings Using the Encoder
# -------------------------------

# Define encoder to include label input
encoder = Model(inputs=[input_features, input_labels], outputs=encoded_output)

# Get the embeddings
X_train_encoded = encoder.predict([X_train_scaled, y_train_onehot])
X_val_encoded = encoder.predict([X_val_scaled, y_val_onehot])
X_test_encoded = encoder.predict([X_test_scaled, y_test_onehot])

print("Encoded Training Set:", X_train_encoded.shape)
print("Encoded Validation Set:", X_val_encoded.shape)
print("Encoded Test Set:", X_test_encoded.shape)

# -------------------------------
# 7. Visualization Function
# -------------------------------

def plot_embedding(X, y, title):
    plt.figure(figsize=(8,6))
    palette = sns.color_palette("hsv", len(np.unique(y)))
    sns.scatterplot(x=X[:,0], y=X[:,1], hue=y, palette=palette, alpha=0.6)
    plt.title(title)
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.legend(title='Class')
    plt.grid(True)
    plt.show()

# Plot Autoencoder Embeddings
print("Conditional Autoencoder Visualization:")
plot_embedding(X_train_encoded, y_train, "Conditional Autoencoder - Training Set")
plot_embedding(X_val_encoded, y_val, "Conditional Autoencoder - Validation Set")
plot_embedding(X_test_encoded, y_test, "Conditional Autoencoder - Test Set")

# -------------------------------
# 8. Training a Classifier on the Encoded Embeddings
# -------------------------------

# Define Early Stopping for the classifier
classifier_early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define the classifier model
classifier = Sequential([
    Input(shape=(encoding_dim,)),  # encoding_dim = 2
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(2, activation='softmax')  # Output layer with 2 neurons for categorical classification
])

# Compile the classifier
classifier.compile(optimizer=Adam(),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

# Train the classifier
history_classifier = classifier.fit(
    X_train_encoded, y_train_onehot,
    epochs=100,
    batch_size=256,
    shuffle=True,
    validation_data=(X_val_encoded, y_val_onehot),
    callbacks=[classifier_early_stop],
    verbose=1
)

# -------------------------------
# 9. Evaluating the Classifier
# -------------------------------

# Predict probabilities on the test set
y_pred_prob = classifier.predict(X_test_encoded)
# Convert probabilities to class labels (0 or 1)
y_pred = np.argmax(y_pred_prob, axis=1)

# Convert true labels from one-hot to integers
y_true = np.argmax(y_test_onehot, axis=1)

# Calculate accuracy
test_accuracy = accuracy_score(y_true, y_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Generate classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(6,5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Class 0', 'Class 1'],
            yticklabels=['Class 0', 'Class 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


Training set: (33600, 24) (33600,)
Validation set: (7200, 24) (7200,)
Test set: (7200, 24) (7200,)
Scaled features using StandardScaler.
Epoch 1/100


/Users/timothychung/opt/anaconda3/envs/ml2/lib/python3.10/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['features_input', 'labels_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 2), output.shape=(None, 24)

In [5]:
# # -------------------------------
# # 1. Import Necessary Libraries
# # -------------------------------
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt
# import seaborn as sns
# import tensorflow as tf
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, concatenate
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping
# from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# # -------------------------------
# # 2. Data Preparation
# # -------------------------------

# # Load data
# X = np.load('Datasets/kryptonite-24-X.npy')
# y = np.load('Datasets/kryptonite-24-y.npy')
# X = np.round(X)

# # Split into training (70%), validation (15%), and test sets (15%)
# X_train, X_temp, y_train, y_temp = train_test_split(
#     X, y, test_size=0.3, random_state=42, stratify=y
# )
# X_val, X_test, y_val, y_test = train_test_split(
#     X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
# )

# print("Training set:", X_train.shape, y_train.shape)
# print("Validation set:", X_val.shape, y_val.shape)
# print("Test set:", X_test.shape, y_test.shape)

# # -------------------------------
# # 3. Feature Scaling
# # -------------------------------

# # Standardize features to have zero mean and unit variance
# scaler = StandardScaler()
# scaler.fit(X_train)

# X_train_scaled = scaler.transform(X_train)
# X_val_scaled = scaler.transform(X_val)
# X_test_scaled = scaler.transform(X_test)

# print("Scaled features using StandardScaler.")

# # -------------------------------
# # 4. Conditional Autoencoder Setup
# # -------------------------------

# # One-hot encode labels
# y_train_onehot = tf.keras.utils.to_categorical(y_train, num_classes=2)
# y_val_onehot = tf.keras.utils.to_categorical(y_val, num_classes=2)
# y_test_onehot = tf.keras.utils.to_categorical(y_test, num_classes=2)

# # Define input dimensions
# input_dim = X_train_scaled.shape[1]
# encoding_dim = 2  # For 2D visualization

# # Modify input to include label information
# input_features = Input(shape=(input_dim,), name='features_input')
# input_labels = Input(shape=(2,), name='labels_input')  

# # Concatenate features and labels
# merged_input = concatenate([input_features, input_labels])

# # Encoder
# encoded = Dense(64, activation='relu')(merged_input)
# encoded = BatchNormalization()(encoded)
# encoded = Dropout(0.2)(encoded)

# encoded = Dense(32, activation='relu')(encoded)
# encoded = BatchNormalization()(encoded)
# encoded = Dropout(0.2)(encoded)

# encoded_output = Dense(encoding_dim, activation='relu', name='encoded_output')(encoded)

# # Decoder: concatenate encoded representation with labels
# decoder_input = concatenate([encoded_output, input_labels])

# decoded = Dense(32, activation='relu')(decoder_input)
# decoded = BatchNormalization()(decoded)
# decoded = Dropout(0.2)(decoded)

# decoded = Dense(64, activation='relu')(decoded)
# decoded = BatchNormalization()(decoded)
# decoded_output = Dense(input_dim, activation='sigmoid', name='decoded_output')(decoded)

# # Classification Head
# classifier = Dense(16, activation='relu')(encoded_output)
# classifier = BatchNormalization()(classifier)
# classifier = Dropout(0.2)(classifier)
# classifier_output = Dense(2, activation='softmax', name='classifier_output')(classifier)

# # Define the combined model
# model = Model(inputs=[input_features, input_labels], outputs=[decoded_output, classifier_output])

# # Compile the model
# # model.compile(optimizer='adam',
# #               loss={'decoded_output': 'binary_crossentropy',
# #                     'classifier_output': 'categorical_crossentropy'},
# #               loss_weights={'decoded_output': 1.0, 'classifier_output': 1.0},
# #               metrics={'classifier_output': 'accuracy'})

# model.compile(optimizer='adam',
#               loss={'decoded_output': 'mean_squared_error',  # Suitable for continuous data in X
#                     'classifier_output': 'categorical_crossentropy'},
#               loss_weights={'decoded_output': 1.0, 'classifier_output': 1.0},
#               metrics={'classifier_output': 'accuracy'})

# # Define Early Stopping
# early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# # -------------------------------
# # 5. Train the Conditional Autoencoder
# # -------------------------------

# history = model.fit(
#     [X_train_scaled, y_train_onehot],  # Inputs: scaled features and one-hot labels
#     {'decoded_output': X_train_scaled, 'classifier_output': y_train_onehot},  # Targets: features and labels
#     epochs=100,
#     batch_size=256,
#     shuffle=True,
#     validation_data=([X_val_scaled, y_val_onehot], {'decoded_output': X_val_scaled, 'classifier_output': y_val_onehot}),
#     callbacks=[early_stop],
#     verbose=1
# )

# # -------------------------------
# # 6. Extract 2D Embeddings Using the Encoder
# # -------------------------------

# # Define encoder to include label input
# encoder = Model(inputs=[input_features, input_labels], outputs=encoded_output)

# # Get the embeddings
# X_train_encoded = encoder.predict([X_train_scaled, y_train_onehot])
# X_val_encoded = encoder.predict([X_val_scaled, y_val_onehot])
# X_test_encoded = encoder.predict([X_test_scaled, y_test_onehot])

# print("Encoded Training Set:", X_train_encoded.shape)
# print("Encoded Validation Set:", X_val_encoded.shape)
# print("Encoded Test Set:", X_test_encoded.shape)

# # -------------------------------
# # 7. Visualization Function
# # -------------------------------

# def plot_embedding(X, y, title):
#     plt.figure(figsize=(8,6))
#     palette = sns.color_palette("hsv", len(np.unique(y)))
#     sns.scatterplot(x=X[:,0], y=X[:,1], hue=y, palette=palette, alpha=0.6)
#     plt.title(title)
#     plt.xlabel('Component 1')
#     plt.ylabel('Component 2')
#     plt.legend(title='Class')
#     plt.grid(True)
#     plt.show()

# # Plot Autoencoder Embeddings
# print("Conditional Autoencoder Visualization:")
# plot_embedding(X_train_encoded, y_train, "Conditional Autoencoder - Training Set")
# plot_embedding(X_val_encoded, y_val, "Conditional Autoencoder - Validation Set")
# plot_embedding(X_test_encoded, y_test, "Conditional Autoencoder - Test Set")

# # -------------------------------
# # 8. Training a Classifier on the Encoded Embeddings
# # -------------------------------

# # Define Early Stopping for the classifier
# classifier_early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# # Define the classifier model
# classifier = Sequential([
#     Input(shape=(encoding_dim,)),  # encoding_dim = 2
#     Dense(16, activation='relu'),
#     Dense(8, activation='relu'),
#     Dense(2, activation='softmax')  # Output layer with 2 neurons for categorical classification
# ])

# # Compile the classifier
# classifier.compile(optimizer=Adam(),
#                    loss='categorical_crossentropy',
#                    metrics=['accuracy'])

# # Train the classifier
# history_classifier = classifier.fit(
#     X_train_encoded, y_train_onehot,
#     epochs=100,
#     batch_size=256,
#     shuffle=True,
#     validation_data=(X_val_encoded, y_val_onehot),
#     callbacks=[classifier_early_stop],
#     verbose=1
# )

# # -------------------------------
# # 9. Evaluating the Classifier
# # -------------------------------

# # Predict probabilities on the test set
# y_pred_prob = classifier.predict(X_test_encoded)
# # Convert probabilities to class labels (0 or 1)
# y_pred = np.argmax(y_pred_prob, axis=1)

# # Convert true labels from one-hot to integers
# y_true = np.argmax(y_test_onehot, axis=1)

# # Calculate accuracy
# test_accuracy = accuracy_score(y_true, y_pred)
# print(f"Test Accuracy: {test_accuracy:.4f}")

# # Generate classification report
# print("\nClassification Report:")
# print(classification_report(y_true, y_pred))

# # Generate confusion matrix
# conf_matrix = confusion_matrix(y_true, y_pred)

# # Plot confusion matrix
# plt.figure(figsize=(6,5))
# sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
#             xticklabels=['Class 0', 'Class 1'],
#             yticklabels=['Class 0', 'Class 1'])
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.title('Confusion Matrix')
# plt.show()

# # -------------------------------
# # 10. (Optional) Visualize Classifier Decision Boundary
# # -------------------------------

# # def plot_decision_boundary(model, X, y, title):
# #     # Define the grid range
# #     x_min, x_max = X[:,0].min() - 1, X[:,0].max() + 1
# #     y_min, y_max = X[:,1].min() - 1, X[:,1].max() + 1
# #     h = 0.01  # step size in the mesh

# #     # Generate a grid of points
# #     xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
# #                          np.arange(y_min, y_max, h))
# #     grid = np.c_[xx.ravel(), yy.ravel()]

# #     # Predict probabilities for each point in the grid
# #     probs = model.predict(grid)
# #     Z = np.argmax(probs, axis=1).reshape(xx.shape)

# #     # Plot contour and training examples
# #     plt.figure(figsize=(10,8))
# #     plt.contourf(xx, yy, Z, levels=[-0.1, 0.5, 1.1], colors=['#FFAAAA', '#AAAAFF'], alpha=0.5)
# #     palette = sns.color_palette("hsv", len(np.unique(y)))
# #     sns.scatterplot(x=X[:,0], y=X[:,1], hue=y, palette=palette, alpha=0.6)
# #     plt.title(title)
# #     plt.xlabel('Component 1')
# #     plt.ylabel('Component 2')
# #     plt.legend(title='Class')
# #     plt.show()

# # # Plot decision boundary on the test set
# # plot_decision_boundary(classifier, X_test_encoded, y_test, "Classifier Decision Boundary on Test Set")


Training set: (33600, 24) (33600,)
Validation set: (7200, 24) (7200,)
Test set: (7200, 24) (7200,)
Scaled features using StandardScaler.
Epoch 1/100


/Users/timothychung/opt/anaconda3/envs/ml2/lib/python3.10/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['features_input', 'labels_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 2), output.shape=(None, 24)